## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
# from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.layers import Flatten, Dense, Dropout
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

import numpy as np

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
batch_size = 50 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 100 # 訓練整個資料集共 100個循環

# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(32,32,3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
FREEZE_LAYERS = 2
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# 輸出整個網路結構
print(net_final.summary())

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0]

In [9]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   horizontal_flip=True)

net_final.fit_generator(train_datagen.flow(x_train, y_train, shuffle=True, batch_size=batch_size),
                        steps_per_epoch=np.trunc(x_train.shape[0]/batch_size),
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test),
                        validation_steps=np.trunc(x_test.shape[0]/batch_size))

score = net_final.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
1000/1000 [==============================] - 90s 90ms/step - loss: 0.0512 - acc: 0.9831 - val_loss: 0.6717 - val_acc: 0.8553
Epoch 2/100
1000/1000 [==============================] - 89s 89ms/step - loss: 0.0454 - acc: 0.9851 - val_loss: 0.6938 - val_acc: 0.8541
Epoch 3/100
1000/1000 [==============================] - 88s 88ms/step - loss: 0.0463 - acc: 0.9853 - val_loss: 0.6855 - val_acc: 0.8560
Epoch 4/100
1000/1000 [==============================] - 88s 88ms/step - loss: 0.0480 - acc: 0.9845 - val_loss: 0.6879 - val_acc: 0.8521
Epoch 5/100
1000/1000 [==============================] - 88s 88ms/step - loss: 0.0445 - acc: 0.9852 - val_loss: 0.6938 - val_acc: 0.8510
Epoch 6/100
1000/1000 [==============================] - 88s 88ms/step - loss: 0.0472 - acc: 0.9844 - val_loss: 0.6791 - val_acc: 0.8565
Epoch 7/100
1000/1000 [==============================] - 88s 88ms/step - loss: 0.0434 - acc: 0.9852 - val_loss: 0.7321 - val_acc: 0.8517
Epoch 8/100
1000/1000 [==================